## Demand Prediction using Machine Learning

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import calendar
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error
import seaborn as sns
%matplotlib inline

In [2]:
df_initial = pd.read_pickle('./DemandDataFile', compression='infer')
df_region = pd.read_pickle('./RegionDataFile', compression='infer')
df_initial = pd.merge(df_initial, df_region, how='inner', right_on=['CITY_NAME'], left_on=['CITY'])
df_initial = df_initial.drop(['CITY_NAME'], axis=1)
df_initial = df_initial[~df_initial['PRODUCT_NAME'].str.contains("Small Flyers|Large Flyers|Meter Bubble Wrap|Bundle of 50 Boxes", na=False)]
df_initial.rename(columns = {'ORDER_DATE':'DATE'},inplace = True)
df_initial.sort_values('DATE',ascending=True, inplace = True)
df_initial.DATE = pd.to_datetime(df_initial['DATE'])

In [3]:
df_fraud = pd.read_csv('./FradulentOrders.csv',dtype={'ORDER_NR': str})

df_initial = df_initial[~df_initial.COD_ORDER_NR.isin(df_fraud.ORDER_NR.tolist())]


In [4]:
df_initial['WareHouse'] = 'Null'
df_initial.loc[:,"WareHouse"][df_initial['REGION_NAME'].isin(['Sindh','Balochistan'])] = 'Karachi'
df_initial.loc[:,"WareHouse"][~df_initial['REGION_NAME'].isin(['Sindh','Balochistan'])] = 'Lahore'
df_initial['WareHouse'].value_counts()

/home/danyal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/danyal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Lahore     1713866
Karachi     977428
Name: WareHouse, dtype: int64

In [5]:
df_initial['MedianPrice'] = df_initial.groupby('SKU')['UNIT_PRICE'].transform('median')

In [6]:
df_initial.head()

,COD_ORDER_NR,SKU,DATE,CATEGORY_LEVEL_1,CATEGORY_LEVEL_2,CATEGORY_LEVEL_3,CATEGORY_LEVEL_4,PRODUCT_NAME,BRAND_NAME,UNIT_PRICE,...,PAYMENT_OPTION,CTV,Gender,CITY,Voucher,MV,Quantity,REGION_NAME,WareHouse,MedianPrice
2865522,304859521,MA521HL1LKRQANAFAMZ,2017-12-01,Home & Living,Kitchen & Dining,Kitchen Tools & Accessories,Kitchen Accessories,Measuring Cups & spoons - Black,MAK TECH,180.0,...,COD,1260971,male,Khairpur,29.9,150.1,1,Sindh,Karachi,180.0
1974837,309169521,IT313OT158UD8NAFAMZ,2017-12-01,Grocer's Shop,Canned & Packaged Foods,Canned & Jarred Food,N/A,Extra Virgin Olive Oil - 250 ml,ITALIA,295.0,...,COD,883217,male,Khanpur mahar,106.2,778.8,3,Sindh,Karachi,335.0
1236288,309141521,SH402EL0Z34CQNAFAMZ,2017-12-01,Phones & Tablets,Phone & Tablet Accessories,Cases & Covers,Android Phones,360 cover For Xiaomi Mi A1 - Black+Red,Shippers,699.0,...,COD,1241380,male,Peshawar,NaN,699.0,1,Khyber Pakhtunkhwa,Lahore,699.0
2571026,303341721,SA609OT148CBONAFAMZ,2017-12-01,Computing & Gaming,Computing,Storage,USB/Flash Drives,Steel Body USB Drive - 32GB,Samsung,1135.0,...,COD,472261,N/A,Nowshera,NaN,1135.0,1,Khyber Pakhtunkhwa,Lahore,1135.0
860063,306815721,PR960HB0ZSCGENAFAMZ,2017-12-01,Beauty & Health,Personal Care,Hair Removal Tools,Waxes,Buy Pro Wax Warmer & Get 100gm Wax Beans + Wax...,PRO WAX 100,1298.0,...,COD,1259830,female,Lahore,NaN,1298.0,1,Punjab,Lahore,1298.0


In [94]:
def compute_shift(df,dateCol,groupCol):
    df['group_no'] = df.groupby([groupCol]).ngroup()
    tmp = df[[dateCol,'Quantity','group_no']].set_index(['group_no',dateCol])\
                                          .unstack('group_no')\
                                          .resample('D').asfreq()
    tmp1 = tmp.shift(1).fillna(0).astype(int).stack('group_no')['Quantity'].rename('D1')
    tmp2 = tmp.shift(2).fillna(0).astype(int).stack('group_no')['Quantity'].rename('D2')
    tmp3 = tmp.shift(3).fillna(0).astype(int).stack('group_no')['Quantity'].rename('D3')
    tmp4 = tmp.shift(4).fillna(0).astype(int).stack('group_no')['Quantity'].rename('D4')
    tmp5 = tmp.shift(5).fillna(0).astype(int).stack('group_no')['Quantity'].rename('D5')
    
    df = df.join(tmp1, on=[dateCol,'group_no'])
    df = df.join(tmp2, on=[dateCol,'group_no'])
    df = df.join(tmp3, on=[dateCol,'group_no'])
    df = df.join(tmp4, on=[dateCol,'group_no'])
    df = df.join(tmp5, on=[dateCol,'group_no'])
    
    df.drop(axis=1, columns=['group_no'], inplace = True)
    return df

In [95]:
def log_inf(x):
    if x>0:
        return np.log(x) 
    else:
        return np.log(1) 

def is_bundle(x):
    if 'Bundle' in x or 'Pack' in x or '+' in x:
        return 1
    else:
        return 0


In [96]:
test = df_initial[df_initial['PRODUCT_NAME'].map(is_bundle) == 1]
test.shape

(401920, 23)

In [97]:
def prepareDataFrame(wareHouse):
    train_df = df_initial[['SKU','DATE','WareHouse','Quantity','MedianPrice','PRODUCT_NAME']][df_initial.WareHouse == wareHouse]
    
    train_df['IsBundle'] = train_df['PRODUCT_NAME'].map(is_bundle)
    #train_df = train_df[(train_df.SKU == 'HP770OT03D0JKNAFAMZ') | (train_df.SKU == 'SH069FA039PJONAFAMZ')]
    train_df = train_df.groupby(by=['SKU','DATE','WareHouse','MedianPrice','IsBundle'], as_index=False)['Quantity'].sum()
    train_df.sort_values('DATE',ascending=True, inplace = True)
    train_df.DATE = pd.to_datetime(train_df['DATE'])
    train_df = train_df.set_index('DATE')

    train_df.head()

    #Gettign the SKUs whcih were not demanded on the start date 
    startDate = '2017-12-01'
    temp = train_df.reset_index().groupby('SKU').first()
    temp.drop(temp[temp.DATE == startDate].index, inplace=True)

    # replacing date to the Min Start date & Quantity Demand to None
    temp['DATE'] = pd.to_datetime(startDate)
    temp['Quantity'] = 0
    if temp.index.name == 'SKU':
        temp.reset_index(inplace = True)
    temp
    temp = temp.set_index('DATE')
    temp
    train_df = train_df.append(temp)
    return train_df

In [ ]:
temp = prepareDataFrame('Karachi')
temp.reset_index(inplace=True)
train_df = compute_shift(temp.copy(),'DATE','SKU')

temp = prepareDataFrame('Lahore')
temp.reset_index(inplace=True)
temp = compute_shift(temp.copy(),'DATE','SKU')
train_df = train_df.append(temp)


In [ ]:
train_df.fillna(0, inplace=True)
train_df.head()

### BenchMark Model (Predict Demand as Avergae of last N days demand)

In [66]:
test_df = train_df[(train_df.DATE >= '2018-05-01') & (train_df.Quantity <= 100)]

In [67]:
test_df['PredictedDemand'] = np.int64((test_df.D1+test_df.D2+test_df.D3+test_df.D4+test_df.D5)/5)
#test_df.loc[:,'PredictedDemand'] = test_df['PredictedDemand'].apply(lambda x : log_inf(x))
test_df.head()

/home/danyal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,DATE,SKU,WareHouse,MedianPrice,IsBundle,Quantity,D1,D2,D3,D4,D5,PredictedDemand
551000,2018-05-01,MO240HB10XKV0NAFAMZ,Karachi,95.0,1,1,0.0,0.0,0.0,0.0,0.0,0
551001,2018-05-01,DA290FAGDTIPNAFAMZ,Karachi,624.0,1,1,0.0,0.0,0.0,0.0,0.0,0
551002,2018-05-01,DM362EL08S8XINAFAMZ,Karachi,249.0,0,1,0.0,0.0,0.0,1.0,0.0,0
551003,2018-05-01,IM505FA01BH2YNAFAMZ,Karachi,300.0,0,1,0.0,0.0,0.0,0.0,0.0,0
551004,2018-05-01,RO070HL0GDBF0NAFAMZ,Karachi,1040.0,0,2,0.0,0.0,0.0,0.0,0.0,0


In [68]:
import math
print("MSE: ",mean_squared_error(test_df.Quantity, test_df.PredictedDemand),
      "RMSE: ",math.sqrt(mean_squared_error(test_df.Quantity, test_df.PredictedDemand))
     )


MSE:  21.006301364908193 RMSE:  4.583263178665196


### ML model Data Prepration

In [69]:
#train_df.drop(axis=1, columns=['Karachi','Lahore'], inplace = True)
dummyWareHouse = pd.get_dummies(train_df['WareHouse'])
dummyWareHouse

train_df = pd.concat([train_df,dummyWareHouse], axis = 1)

In [70]:
from datetime import date
weekNumber = list(map((lambda x: x.isocalendar()[1]), train_df.DATE.dt.date)) 
train_df['WeekNo'] = weekNumber
train_df['WeekDayNo'] = train_df.DATE.dt.weekday

#### **Train Test Data Split

In [71]:
train_df.head()

,DATE,SKU,WareHouse,MedianPrice,IsBundle,Quantity,D1,D2,D3,D4,D5,Karachi,Lahore,WeekNo,WeekDayNo
0,2017-12-01,AU697EL0O2TR8NAFAMZ,Karachi,299.0,0,1,0.0,0.0,0.0,0.0,0.0,1,0,48,4
1,2017-12-01,AL493FA1GB66UNAFAMZ,Karachi,1800.0,0,1,0.0,0.0,0.0,0.0,0.0,1,0,48,4
2,2017-12-01,PA163FA1FDXFINAFAMZ,Karachi,1099.0,1,1,0.0,0.0,0.0,0.0,0.0,1,0,48,4
3,2017-12-01,DE795EL08CH9ENAFAMZ,Karachi,350.0,0,1,0.0,0.0,0.0,0.0,0.0,1,0,48,4
4,2017-12-01,GR678EL1DZ76SNAFAMZ,Karachi,59590.0,0,1,0.0,0.0,0.0,0.0,0.0,1,0,48,4


In [85]:
X = train_df[(train_df.DATE < '2018-05-01') & (train_df.Quantity <= 100)][['D1','D2','MedianPrice','WeekDayNo','Lahore','IsBundle']]
X.MedianPrice = X.MedianPrice.map(log_inf)
Y_orig = train_df[(train_df.DATE < '2018-05-01') & (train_df.Quantity <= 100)][['Quantity']]
#Y = Y_orig.Quantity.map(log_inf)
Y = Y_orig

In [86]:
X_test = train_df[(train_df.DATE >= '2018-05-01') & (train_df.Quantity <= 100)][['D1','D2','MedianPrice','WeekDayNo','Lahore','IsBundle']]
X_test.MedianPrice = X_test.MedianPrice.map(log_inf)
Y_test_orig = train_df[(train_df.DATE >= '2018-05-01') & (train_df.Quantity <= 100)][['Quantity']]
#Y_test = Y_test_orig.Quantity.map(log_inf)
Y_test = Y_test_orig
X_test_SKUs = train_df[(train_df.DATE >= '2018-05-01') & (train_df.Quantity <= 100)][['SKU']]


#### **Decision Tree Regression Model

In [92]:
Y_test.shape
from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(max_depth=7,min_samples_split=3,min_samples_leaf=5)
regr.fit(X, Y)
y_pred = regr.predict(X_test)

import math
print("MSE: ",mean_squared_error((Y_test), (y_pred)),
      "RMSE: ",math.sqrt(mean_squared_error((Y_test), (y_pred)))
     )


MSE:  9.34746804573288 RMSE:  3.0573629234575472


In [80]:
X_test['Actual'] = Y_test
X_test['Predicted'] = y_pred
#X_test['Actual'] = X_test['Actual'].map(np.exp)
#X_test['Predicted'] = X_test['Predicted'].map(np.exp)
#X_test[X_test.Actual > 5].head(100)
#X.MedianPrice.describe()
print('Total Actual Demand : ',X_test.Actual.sum(),'\nTotal Predicted Demand : ',X_test.Predicted.sum())

X_test_SKUs = train_df[(train_df.DATE >= '2018-05-01') & (train_df.Quantity <= 100)][['SKU']]
X_test['SKU'] = X_test_SKUs

X_train_SKUs = train_df[(train_df.DATE < '2018-05-01') & (train_df.Quantity <= 100)][['SKU']]
X['SKU'] = X_train_SKUs

Total Actual Demand :  676646 
Total Predicted Demand :  630510.6903750747


In [81]:
from sklearn.tree import export_graphviz
import os
import subprocess

def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [82]:
features = ['D1','D2','MedianPrice','WeekDayNo','Lahore','IsBundle']
visualize_tree(regr, features)

In [83]:

main_list = list(set(X_test.SKU.unique())-set(X.SKU.unique()))
len(main_list)



0

### GridSearch For Best Model 

In [38]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth': range(4,8,1),'min_samples_leaf':[1, 2, 4, 6, 8, 10]
             ,'min_samples_split' : [2,4,6,8,10,12,14]} 
clf_tree = DecisionTreeRegressor() 

clf = GridSearchCV(clf_tree,parameters)
clf.fit(X,Y)    
clf.best_params_
clf.best_estimator_

{'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 10}

In [44]:
regr = clf.best_estimator_
regr.fit(X, Y)
y_pred = regr.predict(X_test)